In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from tensorflow.python.client import device_lib
def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]
print(get_available_devices()) 

['/device:CPU:0', '/device:GPU:0']


In [ ]:
!pip install hazm
!pip install transformers

import numpy as np
import pandas as pd
from hazm import *
import collections
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Input, Embedding, Dropout,MaxPool1D 
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.utils import to_categorical
from keras.optimizers import Adam,RMSprop
from transformers import TFBertModel,  BertConfig, BertTokenizerFast
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from sklearn.model_selection import train_test_split
from transformers import logging as hf_logging
import logging

tf.get_logger().setLevel(logging.ERROR)
hf_logging.set_verbosity_error()

     |████████████████████████████████| 316 kB 30.8 MB/s 
     |████████████████████████████████| 233 kB 59.8 MB/s 
     |████████████████████████████████| 1.4 MB 47.5 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394487 sha256=1550099d5f23c22c2318a7a6ce8b3885f765386d4abb1402ce750130db022a34
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154056 sha256=a056b7b904e90d6560dcb8d5a5274c8bd6f7b75233f16257fa99fec8a532fba4
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
     |████████████████████████████████| 2.5 MB 16.5 MB/s 
     |████████████████████████████████| 3.3 MB 51.1 MB/s 
    

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/Deep/NLP/persian_news/fixed_train.csv", sep='	')
test = pd.read_csv("/content/drive/MyDrive/Deep/NLP/persian_news/fixed_test.csv", sep='	')
dev = pd.read_csv("/content/drive/MyDrive/Deep/NLP/persian_news/fixed_dev.csv", sep='	')

train=train.append(dev)
train.head()

,Unnamed: 0,content,label,label_id
0,0,گزارش خبرنگار حوزه بهداشت درمان گروه علمی پزشک...,پزشکی,7
1,1,گزارش خبرنگار فوتبال فوتسال گروه ورزشی باشگاه ...,ورزشی,6
2,2,بهروز اکرمی، گفتگو خبرنگار اجتماعی باشگاه خبرن...,اجتماعی,0
3,3,گزارش خبرنگار حوزه شهری گروه اجتماعی باشگاه خب...,اجتماعی,0
4,4,گزارش باشگاه خبرنگاران نقل روابط عمومی تماشاخا...,فرهنگی هنری,5


In [ ]:
np.unique(train['label'])

array(['اجتماعی', 'اقتصادی', 'بین الملل', 'سیاسی', 'علمی فناوری',
       'فرهنگی هنری', 'ورزشی', 'پزشکی'], dtype=object)

In [ ]:
# https://github.com/hooshvare/parsbert

model_name = 'HooshvareLab/bert-fa-base-uncased'

config = BertConfig.from_pretrained(model_name)
config.output_hidden_states = False

tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = model_name, config = config)

transformer_model = TFBertModel.from_pretrained(model_name, config = config)

In [ ]:
max_length = 100

y_train = to_categorical(train['label_id'])

x = tokenizer(
    text=train['content'].to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = False,
    verbose = True)

In [ ]:
bert = transformer_model.layers[0]

input_ids = Input(shape=(max_length,), name='input_ids', dtype='int32')
inputs = {'input_ids': input_ids}

bert_model = bert(inputs)[1]
dropout = Dropout(config.hidden_dropout_prob)
pooled_output = dropout(bert_model, training=False)

output = Dense(units=len(train.label_id.value_counts()), kernel_initializer=TruncatedNormal(stddev=config.initializer_range))(pooled_output)

model = Model(inputs=inputs, outputs=output)


optimizer = Adam(
    learning_rate=5e-05,
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)

model.compile(optimizer = optimizer,loss = CategoricalCrossentropy(from_logits = True), metrics = CategoricalAccuracy('accuracy'))

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 100)]             0         
_________________________________________________________________
bert (TFBertMainLayer)       TFBaseModelOutputWithPool 162841344 
_________________________________________________________________
dropout_37 (Dropout)         (None, 768)               0         
_________________________________________________________________
dense (Dense)                (None, 8)                 6152      
Total params: 162,847,496
Trainable params: 162,847,496
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# input_ids = sequence tokens !
history = model.fit(
    x={'input_ids': x['input_ids']},
    y=y_train,
    validation_split=0.2,
    batch_size=64,
    epochs=10)

Epoch 1/10
185/185 [==============================] - 258s 1s/step - loss: 0.1680 - accuracy: 0.9621 - val_loss: 0.1064 - val_accuracy: 0.9709
Epoch 2/10
185/185 [==============================] - 257s 1s/step - loss: 0.0536 - accuracy: 0.9859 - val_loss: 0.1010 - val_accuracy: 0.9676
Epoch 3/10
185/185 [==============================] - 256s 1s/step - loss: 0.0269 - accuracy: 0.9941 - val_loss: 0.0920 - val_accuracy: 0.9743
Epoch 4/10
185/185 [==============================] - 257s 1s/step - loss: 0.0164 - accuracy: 0.9974 - val_loss: 0.0916 - val_accuracy: 0.9753
Epoch 5/10
185/185 [==============================] - 256s 1s/step - loss: 0.0113 - accuracy: 0.9981 - val_loss: 0.0923 - val_accuracy: 0.9750
Epoch 6/10
185/185 [==============================] - 256s 1s/step - loss: 0.0089 - accuracy: 0.9990 - val_loss: 0.0930 - val_accuracy: 0.9753
Epoch 7/10
185/185 [==============================] - 257s 1s/step - loss: 0.0070 - accuracy: 0.9992 - val_loss: 0.0970 - val_accuracy: 0.9750

In [ ]:
y_test = to_categorical(test['label_id'])

x_test = tokenizer(
    text=test['content'].to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = False,
    verbose = True)

In [ ]:
# Evaluate model
loss_blstm, acc_blstm = model.evaluate({'input_ids': x_test['input_ids']}, y_test)
print('Test Accuracy: %f' % (acc_blstm*100))

52/52 [==============================] - 11s 219ms/step - loss: 0.1042 - accuracy: 0.9745
Test Accuracy: 97.445256


In [ ]:
pred=model.predict({'input_ids': x_test['input_ids']})

In [ ]:
for i in range(10):
  print(test['content'][i])
  print(test['label_id'][i])
  print(np.argmax(pred[i]))

حسن جوهرچی بازیگر سینما تلویزیون ایران گفتگو خبرنگار حوزه سینما گروه فرهنگی باشگاه خبرنگاران جوان؛ خصوص علت کاری اش سال نظر بودجه کافی تولید بازیگران کار می شوند وضعیت بودجه بهبود رود تولیدات قدیم افزایش حرکت جدیدی اتفاق بیفتد دائم سرکار خصوص هوای سینما ایران بیان نظر سینما کم کم تعطیل است، آثار سینمایی مخاطب جذب نمی کند، سالن ها خالی فیلمسازان کاری فعالیت نمی کنند تاسف دارد، مسئولان سینمایی فکری موضوع خصوص تولید فیلم نامه سینما درخشان بازگردد ادامه بحث فیلم نامه موضوع تولید ساختار فیلم سینمای قدیم الایام مشکل صورتی اواخر دهه شصت اوایل دهه هفتاد آثار خوبی سینما می شد حرکت ادامه نکرد جوهرچی بیان دهه های مذکور فیلم های خوبی ناخدا خورشید، اجاره نشین ها ساختیم دلیل فیلم نامه نویسان فراغ بال بهتری کار می کردند ممیزی های گوناگون بحث فیلم نامه خودسانسوری  شروع موضوع تولید تلویزیون سال ها فیلم نامه ها خوبی تلویزیون مکه دست سینما بسته نیمه دهه هفتاد سینمای سیر نزولی می کند جوهرچی اظهار سیر نزولی نیمه دهه هفتاد فیلم های خوبی مادر، کمال الملک پیامد خوبی همراه موضوع مهم توجه فکری حالش بازیگر سینما

اجتماعی = 0 | اقتصادی = 1 | بین الملل = 2 | سیاسی = 3 | علمی فناوری = 4 | فرهنگی هنری = 5 | ورزشی = 6 | پزشکی = 7


In [ ]:
# https://www.farsnews.ir/news/14000428000396

textm='به گزارش حوزه پارلمانی خبرگزاری فارس، آیت الله احمد جنتی دبیر شورای نگهبان صبح امروز (دوشنبه ۲۸ تیر ۱۴۰۰) در جلسه این شورا، با عرض تبریک به مناسبت ایام پر خیر و برکت ماه ذی الحجه و روز عرفه و اعیاد قربان و غدیر گفت: در بهترین ایام سال بعد از ماه مبارک رمضان قرار داریم و از خداوند منان می‌خواهیم که فرصت بهره برداری از فیوضات این روز‌های عزیز را به ما عطا کند. وی روز عرفه را روز عید و بندگی برای مومنان دانست و افزود: بهترین دعا‌های سال، در این ایام به ویژه روز عرفه است که باید از آن بهره کافی و وافی را ببریم.جنتی با گرامیداشت یاد و خاطره شهدای منا، از خداوند خواست تا انتقام خون این شهدا را از حکومت ظالم سعودی بگیرد. وی اعیاد قربان و غدیر را بهترین زمان برای کمک به فقرا عنوان کرد و متذکر شد: از فرصت این روز‌های عزیز همه استفاده کنیم و به ایتام، فقرا و سادات کم بضاعت در حد توان به هر نحوی کمک کنیم تا ان‌شاالله مورد مرضی خداوند قرار گیرد. دبیر شورای نگهبان در بخش پایانی از سخنان خود به معضل آب در خوزستان اشاره و تاکید کرد: با برادران و خواهران ایمانی خود در استان خوزستان ابراز همدردی می‌کنیم و از مسئولان می‌خواهیم با فوریت مشکلات آنان به ویژه مسئله آب را حل کنند و نباید از هیچ کمکی در حل این مسئله دریغ شود؛ بنده و اعضای شورای نگهبان نیز در کنار مردم شریف و عزیز خوزستان هستیم و به اندازه توان و مسئولیت خود برای رفع مشکلات آنان تلاش می‌کنیم.'

x_test2 = tokenizer(
    text=textm,
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = False,
    verbose = True)

pred22=model.predict({'input_ids': x_test2['input_ids']})

print(textm)
print(np.argmax(pred22))

به گزارش حوزه پارلمانی خبرگزاری فارس، آیت الله احمد جنتی دبیر شورای نگهبان صبح امروز (دوشنبه ۲۸ تیر ۱۴۰۰) در جلسه این شورا، با عرض تبریک به مناسبت ایام پر خیر و برکت ماه ذی الحجه و روز عرفه و اعیاد قربان و غدیر گفت: در بهترین ایام سال بعد از ماه مبارک رمضان قرار داریم و از خداوند منان می‌خواهیم که فرصت بهره برداری از فیوضات این روز‌های عزیز را به ما عطا کند. وی روز عرفه را روز عید و بندگی برای مومنان دانست و افزود: بهترین دعا‌های سال، در این ایام به ویژه روز عرفه است که باید از آن بهره کافی و وافی را ببریم.جنتی با گرامیداشت یاد و خاطره شهدای منا، از خداوند خواست تا انتقام خون این شهدا را از حکومت ظالم سعودی بگیرد. وی اعیاد قربان و غدیر را بهترین زمان برای کمک به فقرا عنوان کرد و متذکر شد: از فرصت این روز‌های عزیز همه استفاده کنیم و به ایتام، فقرا و سادات کم بضاعت در حد توان به هر نحوی کمک کنیم تا ان‌شاالله مورد مرضی خداوند قرار گیرد. دبیر شورای نگهبان در بخش پایانی از سخنان خود به معضل آب در خوزستان اشاره و تاکید کرد: با برادران و خواهران ایمانی خود در استان خوزستان ابراز همدردی می‌کنی